# 🚀 Wakeword Training Platform - Google Colab Edition
Use this notebook to train high-performance wakeword models using Colab's free/paid GPUs (T4, L4, A100).

## 1. Platform Setup
Install dependencies and check GPU status.

In [ ]:
!nvidia-smi
!apt-get install -y libsndfile1 ffmpeg
!pip install torch==2.1.2 torchaudio==2.1.2 librosa soundfile pydantic pyyaml wandb optuna gradio

## 2. Project Setup
Clone the repository or mount Google Drive.

In [ ]:
from google.colab import drive
import os

# Optional: Mount Drive to persist models
# drive.mount('/content/drive')

# If you need to clone the repo:
# !git clone https://github.com/your-repo/wakeword-platform.git
# %cd wakeword-platform

## 3. Configuration

In [ ]:
import torch
from src.config.defaults import get_default_config

config = get_default_config()
config.training.batch_size = 128 # Higher for Colab GPUs
config.training.num_workers = 4
config.qat.backend = "qnnpack" # Best for ARM/Mobile targets

## 4. Launch Dashboard
You can run the full UI inside Colab!

In [ ]:
from src.ui.app import launch_app

launch_app(share=True)